# Filometro em mapa

Algumas cidades como São Paulo, Salvador e São Luís criaram um "filometro" para mostrar como andam as filas nos diversos postos de vacinação para covid.

https://deolhonafila.prefeitura.sp.gov.br/  
https://filometro.saude.salvador.ba.gov.br/  
http://semit.saoluis.ma.gov.br/filometro/  

Seria interessante colocar estas informações num mapa, para cada pessoa verificar quais são as opções mais perto dela para se vacinar evitando aglomerações.

Inspiração:

https://www.alphr.com/make-custom-route-google-maps/  
https://www.youtube.com/watch?v=W0L2DW-6HXs  

## Scraping

- scraping -> pandas (selenium)

1. scraping -> pandas df
6. fazer uma prévia no thexs mapping
2. endereços -> get geolocation (precisa ou o google faz sozinho?)
3. insert geolocation in google maps, my maps?
4. colocar os pointers coloridos de acordo com a situação
5. modalidade entra como um filtro lateral de clique, como no the-xs mapping


3. talvez get sp geodata com geobr? Talvez nem precise
4. mapa - ver labcidade, ver api google maps... algo interativo. D3.js ??
5. mapa update situação de 1 em 1 hora

a ideia é ser reprodutível para outros estados

# Instalações e Importações

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

In [1]:
from selenium import webdriver
import time
import pandas as pd
from datetime import datetime

# Scraping

In [93]:
#launch url
url = "https://deolhonafila.prefeitura.sp.gov.br/"

def get_info_sp(url, save_to_csv=True):
    
    #### SCRAPING ####
    
    # Initialize browser driver
    
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver', options=chrome_options)
    
    # opening URL
    driver.get(url)
    time.sleep(5)
    
    # list of lists with 5 elements each
    vaccination_places_list = []
    
    all_locations = driver.find_elements_by_css_selector(".msg")

    for vaccination_place in all_locations:

        name = vaccination_place.find_element_by_xpath('.//h6[1]').get_attribute('innerText')
        modality = vaccination_place.find_element_by_xpath('.//h6[2]').get_attribute('innerText')
        adress = vaccination_place.find_element_by_xpath('.//h11[1]').get_attribute('innerText')
        queue_situation = vaccination_place.find_element_by_xpath('.//h11[2]').get_attribute('innerText')
        last_update = vaccination_place.find_element_by_xpath('.//h11[3]').get_attribute('innerText')

        vaccination_place_info = [name, modality, adress, queue_situation, last_update]

        vaccination_places_list.append(vaccination_place_info)

    # close browser
    driver.quit()
    
    
    #### TRANSFORMING LIST INTO PANDAS DATAFRAME ####

    vaccination_places_df = pd.DataFrame(vaccination_places_list)
    vaccination_places_df.columns = ['nome', 'modalidade', 'endereco', 'situacao_fila', 'ultima_atualizacao']
    
    # data cleaning and formatting
    for column in ('modalidade', 'situacao_fila', 'ultima_atualizacao'):
        vaccination_places_df[column] = [item.split(": ")[1] for item in vaccination_places_df[column]]
    
    vaccination_places_df['ultima_atualizacao'] = pd.to_datetime(vaccination_places_df['ultima_atualizacao'])
    
    # saving to csv
    if save_to_csv:
        city = 'sp_'
        timestamp = datetime.now().strftime('%Y-%m-%d_%H:%M:%S')
        path = city +'vac_' + timestamp + '.csv'
        vaccination_places_df.to_csv(path, sep=";")
    
    return vaccination_places_df
    
    
df = get_info_sp(url)
# vaccination_places_df.index.size # == 516

In [94]:
df.shape

(516, 5)

Temos 516 Postos de Vacinação

In [95]:
df.head()

,nome,modalidade,endereco,situacao_fila,ultima_atualizacao
0,UBS VILA TEREZINHA / CRECHE BATUIRA,POSTO VOLANTE,"R. Jorge Píres Ramalho, 70 - Vila Isabel, São ...",SEM FILA,2021-06-18 15:18:00
1,UBS CASA VERDE / PARÓQUIA NOSSA SENHORA DAS DORES,POSTO VOLANTE,"R. ANTONIETA, 19 - CASA VERDE BAIXA - CEP: 025...",SEM FILA,2021-06-18 15:02:00
2,POSTO VOLANTE CENTER NORTE,POSTO VOLANTE,"Rua Otto Baumgart, 500",SEM FILA,2021-06-18 14:55:00
3,IGREJA DE SÃO JUDAS,POSTO VOLANTE,"Alameda Guaiós,145",FILA GRANDE,2021-06-18 15:50:00
4,SHOPPING METRÔ SANTA CRUZ,POSTO VOLANTE,Rua Domingos de Morais (acesso pela Externa),FILA MÉDIA,2021-06-18 16:05:00


Verificação de nulos

In [96]:
vaccination_places_df.isna().sum()

nome                  0
modalidade            0
endereco              0
situacao_fila         0
ultima_atualizacao    1
dtype: int64

In [97]:
df[df['ultima_atualizacao'].isnull()]

,nome,modalidade,endereco,situacao_fila,ultima_atualizacao
36,MEGA POSTO CENTRO CULTURAL GRAJAÚ,MEGAPOSTO,"RUA PROFESSOR OSCAR BARRETO FILHO, 252 - PARQU...",,NaT


In [89]:
df[df.situacao_fila == '']

,nome,modalidade,endereco,situacao_fila,ultima_atualizacao
36,MEGA POSTO CENTRO CULTURAL GRAJAÚ,MEGAPOSTO,"RUA PROFESSOR OSCAR BARRETO FILHO, 252 - PARQU...",,NaT


<div class="alert alert-success"> Temos 4 situações de fila, com um único valor nulo = '', destacado acima</div>

In [74]:
df.situacao_fila.unique()

array(['SEM FILA', 'FILA MÉDIA', 'FILA PEQUENA', 'FILA GRANDE', ''],
      dtype=object)

<div class="alert alert-success">Também temos 4 tipos de postos de vacinação</div>

In [75]:
df.modalidade.unique()

array(['POSTO VOLANTE', 'DRIVE-THRU', 'MEGAPOSTO', 'POSTO FIXO'],
      dtype=object)

<div class="alert alert-success">Sobre a variação de tempo, parece ser a cada x horas</div>

In [98]:
df.ultima_atualizacao.max() - df.ultima_atualizacao.min()

Timedelta('0 days 02:10:00')

In [99]:
df.ultima_atualizacao.sort_values()

417   2021-06-18 14:45:00
395   2021-06-18 14:46:00
331   2021-06-18 14:47:00
286   2021-06-18 14:47:00
475   2021-06-18 14:47:00
              ...        
364   2021-06-18 16:55:00
51    2021-06-18 16:55:00
247   2021-06-18 16:55:00
161   2021-06-18 16:55:00
36                    NaT
Name: ultima_atualizacao, Length: 516, dtype: datetime64[ns]